# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## (**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

我们现在可以将存储在文件中的数据读回内存。


In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以[**存储一个张量列表，然后把它们读回内存。**]


In [3]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。


In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）确实有用，
但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数。
让我们从熟悉的多层感知机开始尝试一下。


In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]


In [6]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)


In [7]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。


In [8]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。

## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？
1. 如何同时保存网络架构和参数？需要对架构加上什么限制？


[Discussions](https://discuss.d2l.ai/t/1839)


即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？

解答：

  1. 加速模型训练：存储模型参数可以避免每次重新训练模型时需要重复计算之前已经计算过的权重和偏置。

  2. 节省内存空间：保存模型参数比保存完整的模型文件更加节省内存空间，这在处理大型模型或使用内存受限设备时尤为重要。

  3. 便于共享和复现：存储模型参数可以方便地共享和复现已经训练好的模型，其他人可以直接加载这些参数并使用它们进行预测或微调。

  4. 便于调试和分析：通过检查模型参数，可以更容易地诊断模型中存在的问题，并对其进行调整和优化。

In [20]:
import torch
from torch import nn
from torch.nn import functional as F
from collections import OrderedDict


class MLP(nn.Module):  # 定义 MLP 类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 定义隐藏层层，输入尺寸为 20，输出尺寸为 256
        self.output = nn.Linear(256, 10)  # 定义输出层，输入尺寸为 256，输出尺寸为 10

    def forward(self, x):  # 定义前向传播函数
        return self.output(
            F.relu(self.hidden(x))
        )  # 使用 ReLU 激活函数，计算隐藏层和输出层的输出


class MLP_new(nn.Module):  # 定义 MLP 类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 定义隐藏层层，输入尺寸为 20，输出尺寸为 256
        self.output = nn.Linear(256, 10)  # 定义输出层，输入尺寸为 256，输出尺寸为 10

    def forward(self, x):  # 定义前向传播函数
        return self.output(
            F.relu(self.hidden(x))
        )  # 使用 ReLU 激活函数，计算隐藏层和输出层的输出


net = MLP()  # 创建 MLP 的实例
# torch.save(net.hidden.state_dict(), "mlp.hidden.params")  # 将隐藏层的参数保存到文件中
torch.save(net.state_dict(), "mlp.params")  # 将整个 MLP 的参数保存到文件中
clone = MLP_new()  # 创建另一个 MLP 的实例
params = torch.load("mlp.params")  # 加载已保存的参数
print(params.keys())  # 输出参数字典的键，查看包含哪些参数
new_params = OrderedDict(
    {k: v for k, v in params.items() if k.startswith("hidden")}
)  # 只取出隐藏层的参数
print(new_params.keys())  # 输出新参数字典的键，查看包含哪些参数

new_model_dict=clone.state_dict()  # 获取克隆实例的参数字典
new_model_dict.update(new_params)  # 更新克隆实例的参数字典，只包含隐藏层的参数
clone.load_state_dict(new_model_dict)  # 加载更新后的参数

print(
    clone.hidden.weight == net.hidden.weight
)  # 比较两个 MLP 实例的隐藏层权重是否相等，并输出结果

odict_keys(['hidden.weight', 'hidden.bias', 'output.weight', 'output.bias'])
odict_keys(['hidden.weight', 'hidden.bias'])
tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])


如何同时保存网络架构和参数？需要对架构加上什么限制？

解答：

  在PyTorch中，可以使用torch.save()函数同时保存网络架构和参数。为了保存网络架构，需要将模型的结构定义在一个Python类中，并将该类实例化为模型对象。此外，必须确保该类的构造函数不包含任何随机性质的操作，例如dropout层的随机丢弃率应该是固定的。

In [21]:
import torch
from torch import nn
from torch.nn import functional as F


class MLP(nn.Module):  # 定义 MLP 类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 定义隐藏层层，输入尺寸为 20，输出尺寸为 256
        self.output = nn.Linear(256, 10)  # 定义输出层，输入尺寸为 256，输出尺寸为 10

    def forward(self, x):  # 定义前向传播函数
        return self.output(
            F.relu(self.hidden(x))
        )  # 使用 ReLU 激活函数，计算隐藏层和输出层的输出


net = MLP()

# 存储模型
torch.save(net.state_dict(), "model.pt")

# 导入模型
model = torch.load("model.pt")
model

OrderedDict([('hidden.weight',
              tensor([[-0.2095,  0.1770,  0.0797,  ...,  0.1109,  0.2000,  0.0592],
                      [-0.0994, -0.2184,  0.0044,  ...,  0.1986,  0.0438, -0.1765],
                      [ 0.0687,  0.2009, -0.0650,  ...,  0.1985,  0.1109,  0.0996],
                      ...,
                      [-0.1450, -0.0409, -0.2229,  ...,  0.0962, -0.0852,  0.0895],
                      [-0.1965,  0.0946, -0.2063,  ..., -0.1918, -0.0526, -0.1048],
                      [ 0.1362, -0.0733, -0.0673,  ..., -0.1634, -0.0528,  0.0993]])),
             ('hidden.bias',
              tensor([-0.1575,  0.0742, -0.0903, -0.2047, -0.0717, -0.2089, -0.1728, -0.0802,
                       0.1011, -0.2177, -0.1281,  0.1474, -0.2072, -0.2036,  0.1465, -0.0629,
                       0.1974,  0.1116,  0.2170,  0.0650, -0.1202,  0.1384,  0.1374, -0.2117,
                       0.1553, -0.2096, -0.0081,  0.1568, -0.0516,  0.1212, -0.0654, -0.0671,
                       0.1205,